<a href="https://colab.research.google.com/github/AaryanBairagi/GeekForGeeks-21Days-21Projects-Challenge/blob/main/Day_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import sqlite3

In [23]:
##CREATED MY 3 DATASETS AND GENERATED THEIR CSV FILES, GOT THESE LINKS FROM MOCKAROO
! curl "https://api.mockaroo.com/api/62be7000?count=1000&key=d458dec0" > "customers.csv"
! curl "https://api.mockaroo.com/api/ad55cfc0?count=1000&key=d458dec0" > "orders.csv"
! curl "https://api.mockaroo.com/api/492b5160?count=1000&key=d458dec0" > "products.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   99k    0   99k    0     0  57453      0 --:--:--  0:00:01 --:--:-- 57477
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 79512    0 79512    0     0  19934      0 --:--:--  0:00:03 --:--:-- 19932
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3173k    0 3173k    0     0  1053k      0 --:--:--  0:00:03 --:--:-- 1054k


In [24]:
##SETTING UP ALL THE DATAFRAMES
customers = pd.read_csv('customers.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

In [25]:
##CREATE A SQLITE DATABASE AND STORE THEE TABLES FROM MOCKAROO
db_name = "ecommerce.db"
conn = sqlite3.connect(db_name)

customers.to_sql("customers" , conn , index=False , if_exists="replace")
orders.to_sql("orders" , conn , index=False , if_exists="replace")
products.to_sql("products" , conn , index=False , if_exists="replace")

1000

In [26]:
##TESING THE CONNECTION
pd.read_sql("SELECT * FROM customers LIMIT 5;",conn)

,customer_id,first_name,last_name,email,phone_number,address,city,country,postal_code,membership_level
0,1,Ardine,Rodolfi,arodolfi0@godaddy.com,501-847-6534,36444 Holmberg Pass,Šentilj v Slov. Goricah,Slovenia,9241,Basic
1,2,Anastasie,Wombwell,awombwell1@quantcast.com,484-272-9802,54 Prairie Rose Pass,Chajarí,Argentina,3228,Silver
2,3,Dodie,Kaine,dkaine2@seesaa.net,321-671-1353,7 Hudson Junction,Orlando,United States,32835,Silver
3,4,Hashim,Teape,hteape3@dmoz.org,562-187-2255,7834 Riverside Court,Grębów,Poland,39-410,Gold
4,5,Cecil,Konerding,ckonerding4@uol.com.br,269-717-5622,53 Porter Hill,Ludwin,Poland,21-075,Basic


In [27]:
##IMPORTED GOOGLE API KEY

In [28]:
!pip install google-genai

In [29]:
from google import genai
from google.colab import userdata

In [30]:
genai_client = genai.Client(api_key=userdata.get('ecommerce_db'))

In [31]:
##DEFINING ALL THE FUNCTIONS
import json
def get_sql_query(genaiClient ,prompt,user_query) :
  contents = f"""
  {prompt}

  Here's the user query in english you need to work on :
  {user_query}
  """
  response = genaiClient.models.generate_content(model='gemini-2.5-flash',contents=contents)
  print(response)

  usage_metadata = response.usage_metadata

  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count : {usage_metadata.thoughts_token_count}")
  print(f"Output Token Count : {usage_metadata.candidates_token_count}")
  print(f"Total Token Count : {usage_metadata.total_token_count}")

  output = json.loads(response.text.replace('```json','').replace('```',''))
  return output

In [38]:
def execute_query(query,db_name='ecommerce.db') :
  conn = None
  try:
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    print(f"\n Executing query on '{db_name}':\n{query}")
    cursor.execute(query)
    results = cursor.fetchall()

    columns = [description[0] for description in cursor.description]

    results_as_dict = [dict(zip(columns,row)) for row in results]
    results_df = pd.DataFrame(results_as_dict)
    print("Query executed successfully.")
    return results_df

  except sqlite3.Error as e:
    print(f"Database error executing query: {e}")
    return None
  except Exception as e:
    print(f"An unexpected error occurred : {e}")
    return None
  finally :
    if conn:
      conn.close()

In [33]:
def text2sql(genaiClient, prompt,user_query) :
  output = get_sql_query(genaiClient,prompt,user_query)
  if(output['status']=='success'):
    results = execute_query(output['response'])
    return results
  return output

In [34]:
prompt = """
### **ROLE**

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### **CONTEXT**

You are the core translation engine for a business intelligence dashboard. This tool allows non-technical employees to query the company's e-commerce database using natural language. The database dialect is always **SQLite**. Your responses will be executed directly on the database.

The database consists of the following three tables:

**`customers` table:**

```sql
CREATE TABLE customers (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    phone_number VARCHAR(50),
    address VARCHAR(50),
    city VARCHAR(50),
    country VARCHAR(50),
    postal_code VARCHAR(50),
    loyalty_points INT
);
```

**`products` table:**

```sql
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    product_name TEXT,
    description TEXT,
    price DECIMAL(10,2),
    discount_percentage DECIMAL(5,2),
    category VARCHAR(50),
    brand TEXT,
    stock_quantity INT,
    color VARCHAR(50),
    size VARCHAR(20),
    weight DECIMAL(5,2),
    dimensions TEXT,
    release_date DATE,
    rating DECIMAL(3,1),
    reviews_count INT,
    seller_name TEXT,
    seller_rating DECIMAL(3,1),
    seller_reviews_count INT,
    shipping_method VARCHAR(20),
    shipping_cost DECIMAL(6,2)
);
```

**`orders` table:**

```sql
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    product_id INT,
    quantity INT,
    unit_price DECIMAL(10,2),
    total_price DECIMAL(10,2),
    order_date DATE,
    shipping_address VARCHAR(255),
    payment_method VARCHAR(20),
    status VARCHAR(20),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
```

-----

### **TASK**

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

1.  **Analyze the User's Query:** Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like `SUM`, `COUNT`, `AVG`), and ordering they are asking for.
2.  **Map to the Schema:** Map the entities from the user's query to the appropriate tables (`customers`, `products`, `orders`) and columns. Determine the necessary `JOIN` operations using `customers.customer_id` and `products.product_id` as foreign keys in the `orders` table.
3.  **Construct the SQLite Query:** Write a clean and efficient `SELECT` statement that is syntactically correct for SQLite. Ensure all table and column names are accurate.
4.  **Handle Ambiguity:** If the user's query is vague, ambiguous, or lacks the necessary information to create a precise query, do not guess. Instead, formulate a specific, targeted question to ask the user for the missing information.

-----

### **CONSTRAINTS**

  * **Read-Only Operations:** You must **ONLY** generate `SELECT` queries. Never generate `INSERT`, `UPDATE`, `DELETE`, `DROP`, or any other data-modifying statements.
  * **Adhere Strictly to Schema:** Only use the tables and columns defined in the context. Do not invent or assume the existence of any other tables or columns.
  * **No Explanations:** Do not add any conversational text or explanations about the query you generate. Your output must strictly follow the specified format.
  * **Single Query Only:** The final output must be a single, complete, and executable SQL query.
  * **Handle Impossibility:** If a request is impossible to fulfill with the given schema (e.g., "Which employee made the most sales?"), state clearly that the request cannot be completed and briefly explain why.

-----

### **EXAMPLES**

**Example 1: Simple Lookup**

  * **User Query:** "Show me all customers who live in Noida"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT * FROM customers WHERE city = 'Noida';"
    }
    ```

**Example 2: Complex Join and Aggregation**

  * **User Query:** "What are the names of the top 3 products with the highest total revenue?"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT T2.product_name FROM orders AS T1 INNER JOIN products AS T2 ON T1.product_id = T2.product_id GROUP BY T2.product_name ORDER BY SUM(T1.total_price) DESC LIMIT 3;"
    }
    ```

**Example 3: Ambiguous Query**

  * **User Query:** "Show me recent orders"
  * **Expected Output:**
    ```json
    {
      "status": "clarification_needed",
      "response": "Could you please define what 'recent' means? For example, 'in the last 7 days', 'this month', or 'since August 2025'."
    }
    ```

**Example 4: Impossible Query**

  * **User Query:** "Which warehouse has the most stock?"
  * **Expected Output:**
    ```json
    {
      "status": "error",
      "response": "I cannot answer this question as the database does not contain information about warehouses."
    }
    ```

-----

### **OUTPUT FORMAT**

Your final response must be a single JSON object with two keys:

1.  `"status"`: A string with one of three possible values: `"success"`, `"clarification_needed"`, or `"error"`.
2.  `"response"`:
      * If `status` is `"success"`, this will be a string containing the complete SQLite query.
      * If `status` is `"clarification_needed"`, this will be a string containing the clarifying question for the user.
      * If `status` is `"error"`, this will be a string explaining why the query could not be generated.
"""

In [39]:
user_query = "What are my most popular products by total quantity sold?"
text2sql(genai_client, prompt, user_query)

sdk_http_response=HttpResponse(
  headers=<dict len=10>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text="""```json
{
  "status": "success",
  "response": "SELECT T2.product_name FROM orders AS T1 INNER JOIN products AS T2 ON T1.product_id = T2.product_id GROUP BY T2.product_name ORDER BY SUM(T1.quantity) DESC;"
}
```"""
      ),
    ],
    role='model'
  ),
  finish_reason=<FinishReason.STOP: 'STOP'>,
  index=0
)] create_time=None model_version='gemini-2.5-flash' prompt_feedback=None response_id='ZHXZaOqgA9adz7IP-MeSuQU' usage_metadata=GenerateContentResponseUsageMetadata(
  cache_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=1009
    ),
  ],
  cached_content_token_count=1009,
  candidates_token_count=73,
  prompt_token_count=1540,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=1540
    ),
  ],
  thoughts_token_count=190,
  total_t

,product_name
0,Kids' Building Blocks
1,Avocado Oil
2,Wrap Jumpsuit
3,Roasted Garlic Butter
4,Nordic Berry Smoothie
...,...
560,Vegetable Curry
561,Thai Peanut Noodles
562,Self-Watering Planters
563,Cinnamon Ice Cream
